# Résolution numérique de l'équation de Schrödinger pour des molécules

Nous avons vu dans le cours différents scénarios où l'équation de Schrödinger a été résolu pour obtenu les fonctions et valeurs propres.

$$
\hat{H} \Psi = E \Psi
$$

Comme il est coutume en chimie quantique, nous utiliserons l'approximation de Born–Oppenheimer ($\Psi = \psi_N \psi_e$) et considèrerons seulement la partie électronique du système. L'Hamiltonien associé aux électrons est donnée par

$$
\hat{H}_e = -\frac{\hbar^2}{2 m_e} \sum_i \nabla_i^2 -\frac{e^2}{4 \pi \epsilon_0} \sum_i \sum_A \frac{Z_A}{r_{iA}} + \frac{e^2}{4 \pi \epsilon_0} \sum_i \sum_{j>i} \frac{1}{r_{ij}}
$$

où le premier, deuxième et troisième termes font respectivement référence aux énergies cinétiques des électrons, l'attraction électron-noyau et la répulsion électron-électron. Pour de petits systèmes à un électron ($\text{H}_2^+$, $\text{HeH}^{2+}$, ...), il est possible d'obtenir les solutions analytiques des fonctions propres et des valeurs propres de l'équation de Schrödinger. La fonction d'onde peut alors être connue pour ces petits systèmes. Pour des systèmes compliqués (deux électrons ou plus!), une approximation de leur fonction d'onde peut être calculé numériquement à l'aide d'approximations. La formulation de ces approximations définit la méthode de calcul. Dans les prochains exemples, nous utiliserons
- Méthode Hartree-Fock (HF)
- Théorie de la fonctionnelle de la densité (DFT)

Ces méthodes sont implémentées dans plusieurs codes et logiciels tels que [Gaussian](https://gaussian.com/), [ORCA](https://orcaforum.kofo.mpg.de/app.php/portal), [Quantum Espresso](https://www.quantum-espresso.org/), [VASP](https://www.vasp.at/) et bien d'autres. Dans ce notebook, nous utiliserons [PySCF](https://pyscf.org/), qui a l'avantage d'être gratuit, open-source et facile d'utilisation.

## Installation de PySCF

La première étape est d'installer les fonctions nécessaires à nos calculs. L'utilisation de PySCF s'avère utile pour traiter des molécules au niveau de la chimie quantique. La cellule suivante installera PySCF si ce dernier n'est pas déjà installé dans votre environement.

In [ ]:
try:
    import pyscf
except ImportError:
    !pip install pyscf

## Importation des fonctions

Ce ne sont pas toutes les fonctions qui seront pertinentes pour ce travail pratique. Du module `pyscf`, nous importons
* `gto`: Pour définir une molécule sur un ensemble de bases gaussiennes (*Gaussian Type Orbitals*);
* `scf`: *Self-consistent field* pour résoudre l'équation de Schrödinger avec la méthode Hartree-Fock;
* `dft`: Pour résoudre l'équation de Schrödinger en considérant la densité électronique au lieu de la fonction d'onde;
* `tools`: Divers outils, nous l'utiliserons pour créer des fichiers `.cube` pour visualiser des orbitales moléculaires.

In [ ]:
from pyscf import gto, scf, dft, tools

## Définition de la molécule

L'étape suivante est de définir la molécule. La variable `xyz` contient les coordonnées (x, y et z) de chaque atome. Nous créons ensuite une molécule avec la fonction `gto.Mole`, qui accepte les coordonnées des atomes, la charge totale, le spin (différence entre le nombre d'électrons alpha et beta) et l'ensemble de base. La commande `mol.build` construit les quantités nécessaires pour les calculs subséquents.

La cellule suivante contruit un objet python pour la molécule du dihydrogène neutre, singulet et avec un ensemble de bases minimal de type Slater STO-3G.

In [ ]:
xyz = """
H  0.000000    0.000000    0.000000
F  0.000000    0.000000    0.950000
"""

mol = gto.Mole(
    atom = xyz,
    charge = 0,
    spin = 0,
    basis = "STO-3G",
    verbose=4
)
mol.build()

Notons que nous utilisons ici un ensemble de base dit *minimal* ou de simple $\zeta$. Ceci veut dire que l'ensemble de bases ne représentent seulement que le minimum des orbitals définis pour un élément. Dans le cas de HF,

| Atome | Orbital(s)        | Nombre |
|-------|-------------------|--------|
| 1 F   | 1s + 2s + 2px + 2py + 2pz | 5      |
| 1 H   | 1s                | 1      |

Six orbitals atomiques sont alors mélangés pour donner six orbitals moléculaires. Ce nombre change lorsque nous utilisons des ensemble de bases avec un $\zeta$ plus élevé (6-31G, 6-311G, ...), en ajoutant des orbitals de polarisation (représentés par * ou (d,p), (df,pd), ...) ou en ajoutant des fonctions diffuses (représentées par un +).

## Calcul en Hartree-Fock

L'appel au module `scf.RHF` rend accessible la fonction d'onde approximée avec la méthode Hartree-Fock, ou plus précisément la solution *restricted Hartree-Fock*. Le mot *restricted* fait référence à la méthode qui force les électrons à être pairés (2 électrons par orbitale moléculaire). D'autres méthodes peuvent être qualifiées de `unrestricted`, où cette contrainte n'est pas imposée. C'est à la ligne `hf.kernel()` où le calcul se produit et que le résultat est affiché.

In [ ]:
hf = scf.RHF(mol)
hf.kernel()

Le résultat de `-98.572808` est l'énergie de la molécule en hartree (1 hartree vaut 627.5 kcal/mol). L'appel à la fonction `analyze` imprime diverses quantités utiles:
* Les orbitals moléculaires (énergie et nombre d'électrons);
* Les charges sur chaque atome;
* Le moment dipolaire de la molécule.

In [ ]:
hf.analyze(verbose=5);

## Calcul avec la théorie de la fonctionelle de la densité

L'appel au module `dft.KS` met en place les éléments pour obtenir la fonction d'onde approximée avec la [DFT Kohn-Sham](https://doi.org/10.1103/PhysRev.140.A1133) (proposition originale de la DFT en 1965). Tout comme nous l'avons fait dans la section Hartree-Fock, la fonction d'onde est *restricted* (RKS), ce qui impose le pairage des électrons. Le choix de la fonctionelle a été arrêtée à [B3LYP](https://doi.org/10.1063/1.464913), qui est une fonctionelle hybride reconnue pour la qualité des résultats obtennus avec les molécules organiques (contenant principalement des atomes de la deuxième rangée du tableau périodique).

In [ ]:
dft = dft.KS(mol)
dft.xc = "b3lyp"
dft.kernel()

Tout comme dans la dernière section, les résultats peuvent être analysés avec la fonction `analyze()`.

In [ ]:
dft.analyze(verbose=5);

## Analyze/visualisation des orbitales moléculaires

Il est possible d'obtenir les coefficients des orbitals moléculaires (MOs) à partir des calculs fait dans les sections précédantes. Nous allons faire la procédure avec les MOs du calcul Hatree-Fock, car ces dernières sont souvent plus exactes pour regarder les différences d'énergies HOMO-LUMO que les orbitals en DFT.

In [ ]:
# Utile pour créer des fichiers .cube pour la visualisation d'orbitales molécules en 3D.
for i in range(hf.mo_coeff.shape[1]):
    tools.cubegen.orbital(mol, f"orbitale_{i+1:02d}.cube", hf.mo_coeff[:,i])